In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 400     # number of variables
num_ineq = 400    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_400-400.csv")

  2%|█▏                                                                            | 15/1000 [00:58<1:05:47,  4.01s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 10%|███████▊                                                                        | 98/1000 [06:06<54:56,  3.66s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 46%|████████████████████████████████████▎                                          | 459/1000 [28:10<32:29,  3.60s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:02:28<00:00,  3.75s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -70.343501         48.109977      1.194081
std       0.527485          4.199574      0.252281
min     -72.033292         36.899955      0.726004
25%     -70.680788         45.155021      1.006243
50%     -70.355254         48.075926      1.259264
75%     -70.002250         51.013367      1.337236
max     -68.535647         62.058183      2.187687
Number of infeasible solution: 1000


## Learnable Rounding

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 13726.68
Epoch 1, Validation Loss: 15.45
Epoch 2, Validation Loss: -3.43
Epoch 3, Validation Loss: -16.84
Epoch 4, Validation Loss: -19.95
Epoch 5, Validation Loss: -22.07
Epoch 6, Validation Loss: -25.29
Epoch 7, Validation Loss: -25.20
Epoch 8, Validation Loss: -24.43
Epoch 9, Validation Loss: -26.48
Epoch 10, Validation Loss: -27.13
Epoch 11, Validation Loss: -30.13
Epoch 12, Validation Loss: -31.23
Epoch 13, Validation Loss: -32.62
Epoch 14, Validation Loss: -32.98
Epoch 15, Validation Loss: -27.87
Epoch 16, Validation Loss: -33.74
Epoch 17, Validation Loss: -35.63
Epoch 18, Validation Loss: -33.92
Epoch 19, Validation Loss: -32.43
Epoch 20, Validation Loss: -36.59
Epoch 21, Validation Loss: -32.18
Epoch 22, Validation Loss: -35.19
Epoch 23, Validation Loss: -37.74
Epoch 24, Validation Loss: -36.10
Epoch 25, Validation Loss: -37.68
Epoch 26, Validation Loss: -38.82
Epoch 27, Validation Loss: -34.38
Epoch 28, Validation Loss: -37.56
Epoch 29, Validation Los

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_400-400.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:15<00:00,  2.30it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -44.298379          0.032404      0.009528
std       0.232836          0.089142      0.022423
min     -44.741982          0.000000      0.002012
25%     -44.440460          0.000000      0.003526
50%     -44.361336          0.000000      0.004116
75%     -44.293545          0.000000      0.006507
max     -43.458989          0.643882      0.255444
Number of infeasible solution: 203


## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5021.31
Epoch 1, Validation Loss: 5.84
Epoch 2, Validation Loss: -8.14
Epoch 3, Validation Loss: -17.01
Epoch 4, Validation Loss: -18.40
Epoch 5, Validation Loss: -23.67
Epoch 6, Validation Loss: -25.98
Epoch 7, Validation Loss: -28.61
Epoch 8, Validation Loss: -31.35
Epoch 9, Validation Loss: -25.37
Epoch 10, Validation Loss: -32.33
Epoch 11, Validation Loss: -24.39
Epoch 12, Validation Loss: -32.31
Epoch 13, Validation Loss: -31.70
Epoch 14, Validation Loss: -36.19
Epoch 15, Validation Loss: -35.86
Epoch 16, Validation Loss: -36.12
Epoch 17, Validation Loss: -33.82
Epoch 18, Validation Loss: -30.85
Epoch 19, Validation Loss: -36.47
Epoch 20, Validation Loss: -35.70
Epoch 21, Validation Loss: -36.51
Epoch 22, Validation Loss: -36.34
Epoch 23, Validation Loss: -33.03
Epoch 24, Validation Loss: -36.77
Epoch 25, Validation Loss: -38.21
Epoch 26, Validation Loss: -35.23
Epoch 27, Validation Loss: -38.40
Epoch 28, Validation Loss: -38.18
Epoch 29, Validation Loss:

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_400-400.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:28<00:00,  2.23it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -43.856931          0.013985      0.009608
std       0.280737          0.055502      0.021913
min     -44.430156          0.000000      0.001997
25%     -44.062661          0.000000      0.003516
50%     -43.880585          0.000000      0.004206
75%     -43.690001          0.000000      0.006601
max     -42.142440          0.522837      0.269831
Number of infeasible solution: 102
